<a href="https://colab.research.google.com/github/dspacks/MySchoolMenuNotebook/blob/main/school_menu_cal_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# My School Menus iCalendar Generator

This notebook generates iCalendar (`.ics`) files for school menus from [MySchoolMenus.com](https://www.myschoolmenus.com/). You can use these files to import your school's breakfast and lunch menus into any standard calendar application.

## How to Use This Notebook

1.  **Run the first code cell** to install the necessary Python libraries.
2.  **Use the "Find Your IDs" cells** to discover the specific IDs for your district, school, and menus.
3.  **Update the "Configuration" cell** with the IDs you found.
4.  **Run the "Generate Calendars" cell** to create your `.ics` files.
5.  **Download your files** using the instructions at the bottom of the notebook.

In [1]:
!pip install requests

## Find Your IDs

### 1. Fetch your URLS

The first URL is the breakfast menu link. The second url is the lunch menu.

In [16]:
#@title Enter Your Menu URLs { run: "auto" }
url1 = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102948" #@param {type:"string"}
url2 = "https://menus.healthepro.com/organizations/2230/sites/14066/menus/102946\"" #@param {type:"string"}

menu_urls = []
if url1:
    menu_urls.append(url1)
if url2:
    menu_urls.append(url2)


In [5]:
import requests

def find_districts(name):
    # Note: This is not an official or documented API endpoint
    url = f"https://myschoolmenus.com/api/config/search?term={name}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for item in data:
            if item.get('resource_type') == 'Organization':
                print(f"District Name: {item.get('name')}, District ID: {item.get('resource_id')}")
    else:
        print(f"Error: {response.status_code}")

#@title Search for your district by name { run: "auto" }
district_name = "2230" #@param {type:"string"}
if district_name:
    find_districts(district_name)


Error: 404


### 2. Find Your Site ID (School ID)

Enter the `District ID` you found in the previous step and run the cell. A list of all schools in that district and their Site IDs will be displayed.

In [1]:
import requests

def find_sites(district_id):
    url = f"https://myschoolmenus.com/api/organizations/{district_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for site in data['data']['sites']:
            print(f"Site Name: {site.get('name')}, Site ID: {site.get('id')}")
    else:
        print(f"Error: {response.status_code}")

#@title Enter your District ID to find your Site ID { run: "auto" }
district_id_for_sites = "" #@param {type:"string"}
if district_id_for_sites:
    find_sites(district_id_for_sites)


Error: 404


### 3. Find Your Menu IDs

Enter your `District ID` and `Site ID` and run the cell to see a list of available menus (e.g., Lunch, Breakfast) and their IDs.

In [5]:
import requests

def find_menus(district_id, site_id):
    url = f"https://myschoolmenus.com/api/organizations/{district_id}/sites/{site_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        for menu in data['data']['menus']:
            print(f"Menu Name: {menu.get('name')}, Menu ID: {menu.get('id')}")
    else:
        print(f"Error: {response.status_code}")

#@title Enter your District and Site ID to find your Menu IDs { run: "auto" }
district_id_for_menus = "" #@param {type:"string"}
site_id_for_menus = "" #@param {type:"string"}
if district_id_for_menus and site_id_for_menus:
    find_menus(district_id_for_menus, site_id_for_menus)


KeyError: 'menus'

## Generate Calendars

In [7]:
import requests
from datetime import datetime, time, timedelta
from dataclasses import dataclass
import json
import uuid

#@title School and Menu Configuration
#@markdown ---
#@markdown ### Add your school and menu information below
school_configs = [
    {
        "district_id": {district_id_for_menus},
        "site_id": {district_id_for_sites}},
        "lunch_menu_id": {lunch_menu_id},
        "breakfast_menu_id": 102948
    }
]

#@markdown ### Combine all schools into a single calendar file?
combine_calendars = False #@param {type:"boolean"}

DOMAIN = 'myschoolmenus.com'


@dataclass
class RequestParams:
    path: str = None
    headers: dict = None
    exception_message: str = None


class Request:
    def __init__(self):
        pass

    @staticmethod
    def url(path) -> str:
        return f"https://{DOMAIN}{path}"

    @staticmethod
    def get(params: RequestParams) -> dict:
        url = f"{Request.url(params.path)}"
        response = requests.get(url=url, headers=params.headers)
        if response.status_code != 200:
            raise ValueError(
                f"Endpoint {url} returned status code {response.status_code}: {response.reason}"
            )
        try:
            json_response = response.json()
            if not json_response.get('data'):
                raise ValueError(params.exception_message)
        except requests.exceptions.JSONDecodeError:
            raise ValueError(f"Unable to decode JSON response")
        return json_response


class Menus:
    def __init__(self):
        self.path = '/api/organizations'

    def get(self, district_id: int, site_id: int = None, menu_id: int = None, date: datetime.date = None) -> dict:
        exception_message = f"No menu found for district {district_id}"
        if site_id:
            path = self.path + f"/{district_id}/sites/{site_id}"
        elif menu_id:
            path = self.path + f"/{district_id}/menus/{menu_id}"
            if date:
                path = path + f"/year/{date.strftime('%Y')}/month/{date.strftime('%m')}/date_overwrites"
        else:
            path = self.path
        return Request.get(RequestParams(
            path=path,
            exception_message=exception_message + f", menu {menu_id}{f', and date {date}' if date else ''}" if menu_id else exception_message
        ))

    @staticmethod
    def menu_months(menu: dict) -> list[datetime.date]:
        return [datetime.fromisoformat(date) for date in menu['data']['published_months']]


class Organizations:
    def __init__(self):
        self.path = '/api/organizations'

    def get(self, organization_id: int = None) -> dict:
        return Request.get(RequestParams(
            path=self.path + f"/{organization_id}" if organization_id else self.path,
            exception_message=f"No organization found for organization {organization_id}" if organization_id else ""
        ))


class Sites:
    def __init__(self):
        self.path = 'api/organizations'

    def get(self, district_id: int, site_id: int) -> dict:
        return Request.get(RequestParams(
            path=self.path + f"/{district_id}/sites/{site_id}",
            exception_message=f"No site found for district {district_id}" + f" and site {site_id}"
        ))

class Calendar:
    def __init__(self, default_breakfast_time=time(8, 0), default_lunch_time=time(12, 0)):
        self.default_breakfast_time = default_breakfast_time
        self.default_lunch_time = default_lunch_time

    def events(self, menu: json, menu_type: str = "lunch", include_time: bool = False) -> list:
        event_list = []
        menu_data = menu['data']
        if not menu_data:
            raise ValueError(
                f"Missing menu data."
            )

        event_time = None
        if include_time:
            if menu_type.lower() == "breakfast":
                event_time = self.default_breakfast_time
            else:
                event_time = self.default_lunch_time

        for entry in menu_data:
            if entry is None:
                continue

            try:
                prefix = "L: " if menu_type.lower() == "lunch" else "B: "
                recipe_count = 0
                category_count = 0
                summary = ""
                description_parts = []

                for item in json.loads(entry['setting'])['current_display']:
                    if item['type'] == 'recipe' and recipe_count == 0:
                        recipe_count += 1
                        summary = f"{prefix}{item['name']}"

                    if item['type'] == 'category' and category_count == 0:
                        category_count += 1
                        description_parts.append(f"{item['name']}:")
                    elif item['type'] == 'category':
                        description_parts.append("")
                        description_parts.append(f"{item['name']}:")
                    else:
                        description_parts.append(item['name'])

                if summary == '':
                    continue

                event = {
                    'summary': summary,
                    'description': description_parts,
                    'uid': str(uuid.uuid4()),
                    'dtstamp': datetime.now(),
                    'transp': 'OPAQUE'
                }

                entry_date = datetime.fromisoformat(entry['day']).date()

                if include_time and event_time:
                    event_datetime = datetime.combine(entry_date, event_time)
                    event['dtstart'] = event_datetime

                    if menu_type.lower() == "breakfast":
                        duration = timedelta(minutes=30)
                    else:
                        duration = timedelta(minutes=45)

                    end_time = event_datetime + duration
                    event['dtend'] = end_time
                else:
                    event['dtstart'] = entry_date

                event_list.append(event)
            except KeyError:
                continue

        return event_list

    def ical(self, events: list) -> str:
        lines = [
            "BEGIN:VCALENDAR",
            "VERSION:2.0",
            "PRODID:-//My School Menus//EN",
            "CALSCALE:GREGORIAN",
            "METHOD:PUBLISH"
        ]

        for event in events:
            lines.append("BEGIN:VEVENT")
            lines.append(f"SUMMARY:{event['summary']}")

            if isinstance(event['dtstart'], datetime):
                dt_str = event['dtstart'].strftime("%Y%m%dT%H%M%S")
                lines.append(f"DTSTART:{dt_str}")
            else:
                dt_str = event['dtstart'].strftime("%Y%m%d")
                lines.append(f"DTSTART;VALUE=DATE:{dt_str}")

            if 'dtend' in event:
                dt_str = event['dtend'].strftime("%Y%m%dT%H%M%S")
                lines.append(f"DTEND:{dt_str}")

            dt_str = event['dtstamp'].strftime("%Y%m%dT%H%M%SZ")
            lines.append(f"DTSTAMP:{dt_str}")
            lines.append(f"UID:{event['uid']}")
            description = "\\n".join(event['description'])
            lines.extend(self._fold_content("DESCRIPTION", description))
            lines.append(f"TRANSP:{event['transp']}")
            lines.append("END:VEVENT")

        lines.append("END:VCALENDAR")
        return "\r\n".join(lines)

    @staticmethod
    def _fold_content(property_name, content):
        first_line = f"{property_name}:{content}"
        if len(first_line) <= 75:
            return [first_line]

        result = []
        current_line = first_line

        while len(current_line) > 75:
            result.append(current_line[:75])
            current_line = " " + current_line[75:]

        if current_line:
            result.append(current_line)

        return result

def generate_ics():
    menus = Menus()
    cal = Calendar(
        default_breakfast_time=time(8, 0),
        default_lunch_time=time(12, 0)
    )

    all_events = []

    for i, config in enumerate(school_configs):
        district_id = config['district_id']
        site_id = config['site_id']
        lunch_menu_id = config.get('lunch_menu_id')
        breakfast_menu_id = config.get('breakfast_menu_id')

        school_events = []

        try:
            if lunch_menu_id:
                lunch_menu = menus.get(district_id=district_id, menu_id=lunch_menu_id, date=datetime.now())
                school_events.extend(cal.events(lunch_menu, menu_type="lunch", include_time=True))
            if breakfast_menu_id:
                breakfast_menu = menus.get(district_id=district_id, menu_id=breakfast_menu_id, date=datetime.now())
                school_events.extend(cal.events(breakfast_menu, menu_type="breakfast", include_time=True))

            if not combine_calendars:
                ical = cal.ical(school_events)
                with open(f'school-{district_id}-{site_id}-menu-calendar.ics', 'w', newline='') as f:
                    f.write(ical)

            all_events.extend(school_events)
            print(f"Successfully generated calendar for school {district_id}-{site_id}")
        except Exception as e:
            print(f"Error generating calendar for school {district_id}-{site_id}: {e}")

    if combine_calendars:
        ical = cal.ical(all_events)
        with open('school-menu-calendar.ics', 'w', newline='') as f:
            f.write(ical)
        print("Combined ICS file generated.")
    else:
        print("Individual ICS files generated.")

generate_ics()


Successfully generated calendar for school 2230-14066
Individual ICS files generated.


## How to Download Your Calendar Files

1.  Click the **folder icon** in the left-hand sidebar to open the file browser.
2.  You should see your generated `.ics` files in the file list.
3.  Click the **three dots** next to the file you want to download and select **Download**.